In [2]:
from bs4 import BeautifulSoup
import requests
import pandas as pd
import csv
import sqlite3 as lite
import math

In [3]:
url = "http://web.archive.org/web/20110514112442/http://unstats.un.org/unsd/demographic/products/socind/education.htm"

In [4]:
r = requests.get(url)

In [5]:
soup = BeautifulSoup(r.content)

In [6]:
table = soup.find_all('table')[6]
table1 = table.find_all('table')[1].find_all('tr', {'class': 'tcont'})

In [7]:
records = []
for rows in table1:
    col = rows.find_all('td')
    country = col[0].string
    year = col[1].string
    total = col[4].string
    men = col[7].string
    women = col[10].string
    record = (country, year, total, men, women)
    records.append(record)
column_name = ['country', 'year', 'total', 'men', 'women']   
UN_data = pd.DataFrame(records, columns = column_name)


In [8]:
UN_data

,country,year,total,men,women
0,Afghanistan,2004,8,11,5
1,Albania,2004,11,11,11
2,Algeria,2005,13,13,13
3,Andorra,2008,11,11,12
4,Anguilla,2008,11,11,11
5,Austria,2008,15,15,15
6,Azerbaijan,2008,13,13,13
7,Bahrain,2006,14,14,15
8,Bangladesh,2007,8,8,8
9,Belarus,2007,15,14,15


In [9]:
con = lite.connect('gdp_data.db')
cur = con.cursor()

In [25]:
query = "DROP TABLE gdp"
cur.execute(query)

In [26]:
with con:
    cur.execute('CREATE TABLE gdp (country_name, _1999, _2000, _2001, _2002, _2003, _2004, _2005, _2006, _2007, _2008, _2009, _2010)')

In [27]:
with open('ny.gdp.mktp.cd_Indicator_en_csv_v2.csv','rU') as inputFile:
    next(inputFile) # skip the first two lines
    next(inputFile)
    header = next(inputFile)
    inputReader = csv.reader(inputFile)
    for line in inputReader:
        with con:
            cur.execute('INSERT INTO gdp (country_name, _1999, _2000, _2001, _2002, _2003, _2004, _2005, _2006, _2007, _2008, _2009, _2010) VALUES ("' + line[0] + '","' + '","'.join(line[43:-5]) + '");')
con.close() # a good practice to close connection to database

In [28]:
con = lite.connect('gdp_data.db')
cur = con.cursor()

In [29]:
gdp_df = pd.read_sql_query("SELECT * FROM gdp ORDER BY country_name",con,index_col='country_name')

In [30]:
gdp_df

,_1999,_2000,_2001,_2002,_2003,_2004,_2005,_2006,_2007,_2008,_2009,_2010
country_name,,,,,,,,,,,,
Afghanistan,,,2461666314.78366,4128818042.25508,4583648921.64112,5285461999.33739,6275076016.47174,7057597614.45076,9843851009.41462,10190534636.6444,12486950468.1773,15936784437.224
Albania,3434402453.29471,3686649386.83962,4091020249.1995,4449373455.28143,5652325081.83146,7464446950.18143,8376483393.94155,9132558096.86514,10701011524.0982,12881353507.8539,12044208085.864,11926957254.6288
Algeria,48640613515.0702,54790058957.3718,54744716706.3542,56760288961.5829,67863832648.2897,85324997369.5015,103198650297.779,117026655313.743,134977867931.582,171000162581.484,137211795394.166,161207304960.455
American Samoa,,,,,,,,,,,,
Andorra,1239840265.63345,1133644294.5425,1264760246.44075,1456198796.29382,1917948475.27513,2322163502.26295,2539759285.92747,2823503852.62328,3245411583.7197,3712034266.50674,,
Angola,6152922942.98032,9129594818.60749,8936079252.77131,12497346042.1583,14188949239.8883,19640862549.9734,28233699333.6996,41789493191.7536,60448887582.7354,84178084135.3609,75492416522.9248,82470894868.3337
Antigua and Barbuda,651851840.339733,783837838.008776,773765171.519999,802529319.160154,839996355.5355,898356132.282607,997374093.496849,1135143572.54523,1289254310.56428,1347349851.85185,1206410370.37037,1135539037.03704
Arab World,634118560209.881,726012626653.227,713007194598.152,724009906745.738,815538537451.947,959405027973.279,1176100113613.77,1399822842753.85,1638668685146.15,2079570727190.08,1791752937396.52,2097884310030.28
Argentina,283523000000,284203750000,268696750000,102040286874.385,129597154256.313,183295704305.31,222907566120.932,264489701813.206,329761480066.765,406003733832.97,378496221140.697,462703793707.187
